### EDA: Train Data

In [3]:
import pandas as pd
import numpy as np

# data preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# exploratory analysis
import matplotlib.pyplot as plt
import mlxtend
from mlxtend.plotting import scatterplotmatrix
from mlxtend.plotting import heatmap
import seaborn as sns
from IPython.display import Image

# model fit
import statsmodels.api as sm
import tensorflow as tf

# ignore warnings (libraries are rapidly changing)
import warnings
warnings.filterwarnings('ignore')

2023-11-11 17:43:14.643064: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-11 17:43:14.837386: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-11 17:43:14.839268: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-11 17:43:16.637170: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
train = pd.read_csv('train_data.csv')

In [5]:
train.head()

,area,year,urban_population,mei,crop_residues,rice_cultivation,drained_organic_soils_co2,net_forest_conversion,fires,manure_mgmt,...,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
0,Afghanistan,1990,-0.225249,0.332179,-0.210388,-0.203216,-0.225647,-0.168273,-0.148050,-0.223085,...,0,0,0,0,0,0,0,0,0,0
1,Albania,1990,-0.250519,0.332179,-0.253341,-0.240589,-0.218329,-0.168273,-0.148131,-0.280909,...,0,0,0,0,0,0,0,0,0,0
2,Algeria,1990,-0.028259,0.332179,-0.222516,-0.241673,-0.225647,-0.162290,-0.143303,-0.192545,...,0,0,0,0,0,0,0,0,0,0
3,American Samoa,1990,-0.271429,0.332179,NaN,-0.189657,-0.225647,-0.168204,-0.148591,NaN,...,0,0,0,0,0,0,0,0,0,0
4,Andorra,1990,-0.271184,0.332179,NaN,-0.189657,-0.225647,-0.168273,-0.148591,NaN,...,0,0,0,0,0,0,0,0,0,0


In [1]:
feat = ['urban_population', 'mei', 'crop_residues', 'rice_cultivation', 'drained_organic_soils_co2', 'net_forest_conversion', 'fires',
            'manure_mgmt', 'food_systems', 'ippu_tot', 'on_farm_energy_use', 'forestland', '5yr_crop_residues', '5yr_rice_cultivation',
            '5yr_drained_organic_soils_co2', '5yr_net_forest_conversion', '5yr_fires', '5yr_manure_mgmt', '5yr_food_systems', '5yr_ippu_tot',
            '5yr_on_farm_energy_use', '5yr_forestland', '10yr_crop_residues', '10yr_rice_cultivation', '10yr_drained_organic_soils_co2',
            '10yr_net_forest_conversion', '10yr_fires', '10yr_manure_mgmt', '10yr_food_systems', '10yr_ippu_tot', '10yr_on_farm_energy_use',
            '10yr_forestland', '20yr_crop_residues', '20yr_rice_cultivation', '20yr_drained_organic_soils_co2', '20yr_net_forest_conversion',
            '20yr_fires', '20yr_manure_mgmt', '20yr_food_systems', '20yr_ippu_tot', '20yr_on_farm_energy_use', '20yr_forestland']
feat_5yr = ['average_temperature_°c', '5yr_crop_residues', '5yr_rice_cultivation','5yr_manure_mgmt', '5yr_food_systems', '5yr_ippu_tot']
feat_10yr = ['average_temperature_°c', '10yr_crop_residues', '10yr_rice_cultivation', '10yr_manure_mgmt', '10yr_food_systems', '10yr_ippu_tot']

In [2]:
#Print correlation matrix
# print(ag_data[variables].corr())

#Print correlation matrix plots
# sns.pairplot(train[features])

scatterplotmatrix(
    train[feat_10yr].values,
    names=feat_10yr,
    figsize=(18,18), alpha=1
);

NameError: name 'scatterplotmatrix' is not defined